In [ ]:
import pandas as pd
import pyarrow.parquet as pq
from sqlalchemy import create_engine
import os
import numpy as np

# ==============================================================================
# 1. CẤU HÌNH THÁNG & KẾT NỐI
# ==============================================================================
TARGET_MONTH = "2024_06"  # <--- SỬA THÁNG Ở ĐÂY
TARGET_MONTH_HYPHEN = TARGET_MONTH.replace("_", "-")

# IP Tailscale của bạn
DB_HOST = "100.117.51.34"

print(f"🚀 SERVER: {DB_HOST} | THÁNG: {TARGET_MONTH}")

# Ops Database (PostgreSQL) - User postgres
DB_OPS_URL = f"postgresql://postgres:hung12345@{DB_HOST}:5433/Uber_ops"
engine_ops = create_engine(DB_OPS_URL)

# CRM Database (MariaDB) - User root
DB_CRM_URL = f"mysql+pymysql://root:hung12345@{DB_HOST}:3307/Uber_crm"
engine_crm = create_engine(DB_CRM_URL)

# ==============================================================================
# 2. ĐỊNH NGHĨA SCHEMA (BẢN ĐỒ CỘT) - "NGUỒN SỰ THẬT"
# ==============================================================================
# Đây là danh sách cột chính xác theo DDL bạn gửi.
# Code sẽ chỉ cho phép các cột này đi vào DB.

DB_SCHEMAS = {
    # --- OPS DB ---
    "vehicles": [
        "vehicle_id", "license_plate", "make_model", "color", "capacity"
    ],
    "drivers": [
        "driver_id", "legal_name", "license_number", "date_of_birth", "driver_status", "vehicle_id"
    ],
    "trips": [
        "trip_id", "driver_id", "customer_id", "vendorid", 
        "tpep_pickup_datetime", "tpep_dropoff_datetime", "passenger_count", 
        "trip_distance", "ratecodeid", "store_and_fwd_flag", 
        "pulocationid", "dolocationid", "payment_type", "fare_amount", 
        "extra", "mta_tax", "tip_amount", "tolls_amount", 
        "improvement_surcharge", "total_amount", "congestion_surcharge", 
        "airport_fee", "cbd_congestion_fee"
    ],
    
    # --- CRM DB ---
    "customers": [
        "customer_id", "display_name", "phone_number", "email", 
        "registration_date", "customer_segment"
    ],
    "promotions": [
        "promotion_id", "promo_code", "description", "discount_value", 
        "discount_type", "start_date", "end_date", "is_active"
    ],
    "driver_performance": [
        "driver_id", "period_date", "total_trips", "total_revenue", 
        "average_rating", "acceptance_rate", "cancellation_rate", 
        "online_hours", "driver_tier"
    ],
    "trip_feedback": [
        "feedback_id", "trip_id", "customer_id", "rating", 
        "feedback_text", "feedback_date", "used_promotion_id"
    ]
}

# Bản đồ đổi tên cột (Parquet -> Database)
# Chỉ cần liệt kê những cột bị lệch tên (Thường là CamelCase sang lowercase)
RENAME_MAPPING = {
    # Trips Mapping
    "VendorID": "vendorid",
    "RatecodeID": "ratecodeid",
    "PULocationID": "pulocationid",
    "DOLocationID": "dolocationid",
    "Airport_fee": "airport_fee",
    
    # Giữ nguyên các cột khác nếu tên đã khớp (vd: payment_type, fare_amount...)
}

# ==============================================================================
# 3. HÀM INSERT THÔNG MINH
# ==============================================================================
def insert_smart(file_path, table_name, engine, chunksize=10000):
    if not os.path.exists(file_path):
        # Fallback tìm tên file gạch dưới/gạch ngang
        if "_" in file_path: alt_path = file_path.replace("_", "-")
        else: alt_path = file_path.replace("-", "_")
        
        if os.path.exists(alt_path):
            file_path = alt_path
        else:
            print(f"⚠️  MISSING: {file_path}")
            return

    print(f"📂 Processing: {os.path.basename(file_path)} -> Table: {table_name}")

    try:
        # 1. Đọc file
        table = pq.read_table(file_path)
        df = table.to_pandas()

        # 2. Đổi tên cột (Fix lỗi Case Sensitivity)
        df.rename(columns=RENAME_MAPPING, inplace=True)
        
        # 3. Fix lỗi thiếu trip_id cho bảng trips
        if table_name == "trips" and "trip_id" not in df.columns:
            print("   🔧 Generating trip_id...")
            # Logic: YYYYMM + 7 số Index
            # Lấy prefix từ tên file hoặc TARGET_MONTH
            try:
                prefix = int(TARGET_MONTH.replace("_", "").replace("-", ""))
            except:
                prefix = 202401 # Default
            
            start_id = prefix * 10_000_000
            df['trip_id'] = range(start_id, start_id + len(df))

        # 4. CHỈ LẤY CÁC CỘT CÓ TRONG SCHEMA DB
        # Bước này cực quan trọng: Nó lọc bỏ cột rác và sắp xếp đúng thứ tự
        if table_name in DB_SCHEMAS:
            valid_cols = DB_SCHEMAS[table_name]
            # Chỉ lấy những cột tồn tại trong cả DF và Schema
            available_cols = [c for c in valid_cols if c in df.columns]
            
            # Cảnh báo nếu thiếu cột quan trọng
            missing_cols = set(valid_cols) - set(df.columns)
            if missing_cols:
                print(f"   ⚠️ Warning: File thiếu cột so với DB: {missing_cols}")
                # (Tùy chọn) Có thể fill NaN cho cột thiếu nếu cần thiết
                # for c in missing_cols: df[c] = None
            
            # Lọc DataFrame
            df = df[available_cols]
        else:
            print(f"   ❌ Error: Không tìm thấy schema cho bảng {table_name}")
            return

        # 5. Insert vào DB
        row_count = len(df)
        print(f"   -> Inserting {row_count:,} rows...")
        
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False,
            chunksize=chunksize,
            method='multi'
        )
        print(f"   ✅ Success!")

    except Exception as e:
        print(f"   ❌ FAILED: {e}")

# ==============================================================================
# 4. THỰC THI
# ==============================================================================

# --- OPS (PostgreSQL) ---
insert_smart(f"Dữ liệu/01_vehicles/vehicles_{TARGET_MONTH}.parquet", "vehicles", engine_ops)
insert_smart(f"Dữ liệu/02_driver/drivers_{TARGET_MONTH}.parquet", "drivers", engine_ops)

# Trips (Thường tên file là gạch ngang do Colab)
insert_smart(f"Dữ liệu/05_Trips/cleaned_yellow_tripdata_{TARGET_MONTH_HYPHEN}.parquet", "trips", engine_ops, chunksize=5000)

# --- CRM (MariaDB) ---
insert_smart(f"Dữ liệu/03_customer/customers_{TARGET_MONTH}.parquet", "customers", engine_crm)
insert_smart(f"Dữ liệu/04_promotions/promotions_{TARGET_MONTH}.parquet", "promotions", engine_crm)
insert_smart(f"Dữ liệu/06_Driver_performance/driver_performance_{TARGET_MONTH_HYPHEN}.parquet", "driver_performance", engine_crm)
insert_smart(f"Dữ liệu/07_TRIP_FEEDBACK/trip_feedback_{TARGET_MONTH_HYPHEN}.parquet", "trip_feedback", engine_crm)

print("\n🎉 DONE ALL.")